# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error
%matplotlib inline


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that reference data
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))

    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']

    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: array of errors between pulse rate and references.
        confidence_est:array of confidence estimates for pulse rate error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estim0tes = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error

    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)

    return AggregateErrorMetric(errs, confs)


def windowned_split(signal, fs, time_wind=8, overlap=6):
    """
    Function to create subsets of signals using windows of specific duration

    Usage:
        signal: singal from which are generated the subsets
        fs: sampling frequency
        time_wind: length of the time window
        overlap: overlap between windows

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
    """

    windows = []
    for n in range(0, fs*signal.shape[0], ((time_wind-overlap)*fs)):
        if len(signal[n:n+(time_wind)*fs]) == (time_wind*fs):
            windows.append(signal[n:n+(time_wind)*fs])

    return windows


def Feats_df(GTruth, PPGs_feat):
    """
    Function that creates dataframe containing features the ground truth

    Usage:
        GTruth: List containing the real users heart rates
        PPGs_feat: List containing users features

    Returns:
        df: Dictionary containing the users features and their real heartrates
    """

    feat = []
    ref = []
    subjects = []
    for n in range(12):
        for x, y in zip(PPGs_feat[n], GTruth[n]):
            ref.append(y[0])
            feat.append(x)
            subjects.append(n+1)

    data_dict = {}

    for n in range(np.array(feat).T.shape[0]):
        data_dict['subject'] = subjects
        data_dict['feat'+str(n)] = np.array(feat).T[n]

    data_dict['HR'] = ref

    return pd.DataFrame(data_dict)


def Train_Regressor(data, ref, fs, Nf=5, twin=8, ove=6, lof=40/60, hif=240/60):
    """
    Function to train the model used to estimate the users heartrates

    Usage:
        data_fls: filename containing the users signals
        ref_fls: filename containing the ground truth for the user heart rate
        fs: sampling frequency
        N_feat: number of frequencies to use as features
        time_wind: length of the time window
        overlap: overlap between windows
        low_cutf: low cutting frequency
        hi_cutf: high cutting frequency

    Returns:
        clf: Trained Randomforest regressor model
    """
    GTruth = []
    ACCs_feat = []
    PPGs_feat = []
    Subjects = []

    for dat, ref in zip(data, ref):

        GTruth.append(sp.io.loadmat(ref)['BPM0'])
        ppg, accx, accy, accz = LoadTroikaDataFile(dat)
        hs_n, hs_d = signal.butter(3, (lof, hif), 'bandpass', fs=fs)
        accs = [accx, accy, accz]
        Features, signals, spectrums = Preprocessing(ppg, accs, fs, Nf=Nf)
        PPGs_feat.append(Features)

    df = Feats_df(GTruth, PPGs_feat)

    clf = Regressor_HR()

    X = df[df.columns[1:(Nf*2)+1]].values
    y = df.HR.values

    clf.fit(X, y)

    return clf


def Preprocessing(ppg, accs, fs, Nf=5, twin=8, over=6, lof=40/60, hif=240/60):
    """
    Retrieve that find features to estimate the users heart rates

    Usage:
        ppg: time series related to the ppg sensor
        accx: time series related to the acceelerometer of the x axis
        accy: time series related to the acceelerometer of the y axis
        accz: time series related to the acceelerometer of the z axis
        fs: sampling frequency
        N_feat: number of frequencies to use as features
        time_wind: length of the time window
        overlap: overlap between windows
        low_cutf: low cutting frequency
        hi_cutf: high cutting frequency

    Returns:
        PPGs: Features generated from the PPG signals
        signals: Time series related to the windows of the ppg signal
        spectrums: Spectrum related to the windows of the ppg signal
    """

    accx, accy, accz = accs[0], accs[1], accs[2]

    # plt.plot(ppg)
    ppg = (ppg - ppg.mean())

    # Estimating the frequency domain representation
    ppg_windows = windowned_split(ppg, fs, twin, over)
    accx_windows = windowned_split(accx, fs, twin, over)
    accy_windows = windowned_split(accy, fs, twin, over)
    accz_windows = windowned_split(accz, fs, twin, over)

    # Defining the band pass filter
    hs_n, hs_d = signal.butter(3, (lof,  hif), 'bandpass', fs=fs)

    #
    PPGs_feat = []
    ACCs_feat = []
    signals = []
    spectrums = []

    for n in range(len(ppg_windows)):

        accx_w = sp.signal.filtfilt(hs_n, hs_d, accx_windows[n])
        accy_w = sp.signal.filtfilt(hs_n, hs_d, accy_windows[n])
        accz_w = sp.signal.filtfilt(hs_n, hs_d, accz_windows[n])
        acc_w = np.array(np.sqrt(accx_w**2+accy_w**2+accz_w**2))
        ppg_window = sp.signal.filtfilt(hs_n, hs_d, ppg_windows[n])
        signals.append(ppg_window)

        from numpy.fft import rfftfreq
        acc_freq = rfftfreq(len(acc_w), 1/fs), np.abs(np.fft.rfft(acc_w))
        acc_freq[1][(acc_freq[0] >= lof) & (acc_freq[0] <= hif)]
        acc_pks = sp.signal.find_peaks(acc_freq[1], height=10, distance=5)[0]

        temp = np.fft.rfft(ppg_window)
        ppg_freq = rfftfreq(len(ppg_window), 1/fs), np.abs(temp)
        ppg_freq[1][(ppg_freq[0] >= lof) & (ppg_freq[0] <= hif)]
        pks = sp.signal.find_peaks(ppg_freq[1], height=100, distance=10)[0]
        temp = (ppg_freq[1][pks].max()) == ppg_freq[1][pks]
        ppg_freq_max = ppg_freq[0][pks][temp]
        spectrums.append(ppg_freq)

        PPGs_feat.append(ppg_freq[0][[ppg_freq[1].argsort()[-Nf:][::-1]]])
        ACCs_feat.append(acc_freq[0][[acc_freq[1].argsort()[-Nf:][::-1]]])

    Feat = [np.concatenate([x, y]) for x, y in zip(PPGs_feat, ACCs_feat)]

    return Feat, signals, spectrums


def Regressor_HR():
    """
    Function that generates a randomforest model as a regressor

    Returns:
        clf: Randomforest regressor model
    """
    # Hyperparameters
    n_estimators = 400
    max_tree_depth = 20

    # Model
    ne = n_estimators
    md = max_tree_depth

    clf = RandomForestRegressor(n_estimators=ne, max_depth=md, random_state=42)

    return clf


def RunPulseRateAlgorithm(data_fl, ref_fl):

    """
    Estimate heart rates

    Usage:
        data_fl: filename containing the users signals
        ref_fl: filename containing the ground truth for the user heart rate

    Returns:
        pr_err: List containing the errors of the predictions
        pr_conf: list containing the confidence of the predictions

    """

    # Interval variables
    fs = 125
    N_feat = 5

    # Loading the data from the users
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    data_train, ref_train = LoadTroikaDataset()

    # Getting features for the predictions
    accs = [accx, accy, accz]
    Features, signals, spectrums = Preprocessing(ppg, accs, fs=fs, Nf=N_feat)
    Features = np.array(Features)

    # Generating the model to find the heart rate
    clf = Train_Regressor(data_train, ref_train, fs=fs, Nf=N_feat)

    # Compute pulse rate estimates and estimation confidence
    pr_est = clf.predict(Features)
    pr_ref = sp.io.loadmat(ref_fl)['BPM0'].reshape(-1)

    # Finding metrics to measure the performance of the model
    pr_conf = []
    pr_err = []

    for n in range(len(signals)):

        freqs = spectrums[n][0]
        hr_f = pr_est[n]/60
        fft_mag = np.abs(spectrums[n][1])
        window_f = 5/60
        win1 = hr_f - window_f
        win2 = hr_f + window_f
        fundamental_freq_wind = (freqs > win1) & (freqs < win2)

        pr_conf.append(np.sum(fft_mag[fundamental_freq_wind])/np.sum(fft_mag))
        pr_err.append(np.abs((pr_est[n]-pr_ref[n])))

    # Return per-estimate mean absolute error and confidence
    # ref_idx = np.cumsum(np.ones(len(ref)) * 125 * 2) - 125 * 2
    return pr_err, pr_conf
    